# RNN_텍스트분석
- RNN : 순환신경망
- 언어, 시계열 데이터 분석에 주로 사용
- 길이가 길어지면 기울기 소실 문제 발생
- LSTM, GPU 등으로 문제 보완

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding
from tensorflow.keras.utils import to_categorical
import pandas as pd
import numpy as np

# Tensorflow에서 텍스트 전처리
- 토큰화 : 문장을 단어 혹은 형태소로 쪼개는 것
- 원핫인코딩 : 문자를 벡터화
- 임베딩(Embedding) : 벡터화 -> 원핫인코딩을 더 축소


In [3]:
text = "해보지 않으면 해낼 수 없다"

In [4]:
result = text_to_word_sequence(text)
result

['해보지', '않으면', '해낼', '수', '없다']

### 단어 빈도수 세기

In [5]:
docs = ['먼저 텍스트의 각 단어를 나누어 토큰화 합니다.',
       '텍스트의 단어로 트큰화해야 딥러닝에서 인식됩니다.',
       '토근화한 결과는 딥러닝에서 사용할 수 있습니다.',
       '텍스트 전처리에는 벡터화 원핫인코딩 패딩으로 길이 맞추기 등이 필요합니다.',
       '딥러닝 쉽지 않네요.']

In [7]:
token = Tokenizer()
token.fit_on_texts(docs)

print('단어 count: ', token.word_counts, end='\n\n')
print('문장 count: ', token.document_count, end='\n\n')
print('각 단어가 몇 개의 문장에 포함되어 있는지 계산: ', token.word_docs, end='\n\n')
print('각 단어에 매겨진 인덱스값: ', token.word_index, end='\n\n')

단어 count:  OrderedDict([('먼저', 1), ('텍스트의', 2), ('각', 1), ('단어를', 1), ('나누어', 1), ('토큰화', 1), ('합니다', 1), ('단어로', 1), ('트큰화해야', 1), ('딥러닝에서', 2), ('인식됩니다', 1), ('토근화한', 1), ('결과는', 1), ('사용할', 1), ('수', 1), ('있습니다', 1), ('텍스트', 1), ('전처리에는', 1), ('벡터화', 1), ('원핫인코딩', 1), ('패딩으로', 1), ('길이', 1), ('맞추기', 1), ('등이', 1), ('필요합니다', 1), ('딥러닝', 1), ('쉽지', 1), ('않네요', 1)])

문장 count:  5

각 단어가 몇 개의 문장에 포함되어 있는지 계산:  defaultdict(<class 'int'>, {'텍스트의': 2, '토큰화': 1, '합니다': 1, '각': 1, '단어를': 1, '나누어': 1, '먼저': 1, '트큰화해야': 1, '딥러닝에서': 2, '단어로': 1, '인식됩니다': 1, '수': 1, '있습니다': 1, '결과는': 1, '사용할': 1, '토근화한': 1, '길이': 1, '패딩으로': 1, '전처리에는': 1, '원핫인코딩': 1, '필요합니다': 1, '텍스트': 1, '등이': 1, '벡터화': 1, '맞추기': 1, '않네요': 1, '쉽지': 1, '딥러닝': 1})

각 단어에 매겨진 인덱스값:  {'텍스트의': 1, '딥러닝에서': 2, '먼저': 3, '각': 4, '단어를': 5, '나누어': 6, '토큰화': 7, '합니다': 8, '단어로': 9, '트큰화해야': 10, '인식됩니다': 11, '토근화한': 12, '결과는': 13, '사용할': 14, '수': 15, '있습니다': 16, '텍스트': 17, '전처리에는': 18, '벡터화': 19, '원핫인코딩': 20, '패딩으로': 21, '길이': 22, '맞추기': 23,

- '딥러닝 쉽지 않네요' : [26, 27, 28]

In [9]:
x = token.texts_to_sequences(docs)
print(x)

[[3, 1, 4, 5, 6, 7, 8], [1, 9, 10, 2, 11], [12, 13, 2, 14, 15, 16], [17, 18, 19, 20, 21, 22, 23, 24, 25], [26, 27, 28]]


### 가장 긴 벡터의 길이 추출

In [10]:
# doc에서 가장 긴 길이를 가진 문장
max([len(i) for i in x])

9

## 가장 긴 길이의 문장에 맞추어 패딩 수행
- 가장 긴 문장 길이 + 1 의 길이로 패딩 진행
- 문장의 시작에는 반드시 0이 있어야 함
    

In [11]:
padded_x = pad_sequences(x, max([len(i) for i in x])+1)
padded_x

array([[ 0,  0,  0,  3,  1,  4,  5,  6,  7,  8],
       [ 0,  0,  0,  0,  0,  1,  9, 10,  2, 11],
       [ 0,  0,  0,  0, 12, 13,  2, 14, 15, 16],
       [ 0, 17, 18, 19, 20, 21, 22, 23, 24, 25],
       [ 0,  0,  0,  0,  0,  0,  0, 26, 27, 28]], dtype=int32)

In [12]:
padded_x.shape

(5, 10)

# 텍스트를 읽고 긍정, 부정 예측


In [21]:
docs2 = ['너무 재미있네요',
        '최고예요',
        '참 신기한 딥러닝이네요',
        '인공지능 칭찬합니다',
        '더 자세히 배우고 싶어요',
        '변화가 너무 빨라요',
        'GPT 성능이 생각보다 별로네요',
        '제미나이보다는 낫죠',
        '나는 차라리 라마를 쓴다',
        '유료 결재 싫어요']

In [22]:
classes = np.array([1,1,1,1,1,0,0,1,0,0])

In [23]:
token = Tokenizer()
token.fit_on_texts(docs2)
print(token.word_index)

{'너무': 1, '재미있네요': 2, '최고예요': 3, '참': 4, '신기한': 5, '딥러닝이네요': 6, '인공지능': 7, '칭찬합니다': 8, '더': 9, '자세히': 10, '배우고': 11, '싶어요': 12, '변화가': 13, '빨라요': 14, 'gpt': 15, '성능이': 16, '생각보다': 17, '별로네요': 18, '제미나이보다는': 19, '낫죠': 20, '나는': 21, '차라리': 22, '라마를': 23, '쓴다': 24, '유료': 25, '결재': 26, '싫어요': 27}


In [24]:
x = token.texts_to_sequences(docs2)
x

[[1, 2],
 [3],
 [4, 5, 6],
 [7, 8],
 [9, 10, 11, 12],
 [13, 1, 14],
 [15, 16, 17, 18],
 [19, 20],
 [21, 22, 23, 24],
 [25, 26, 27]]

In [25]:
max([len(i) for i in x])

4

In [26]:
padding_x = pad_sequences(x, max([len(i) for i in x])+1)
padding_x

array([[ 0,  0,  0,  1,  2],
       [ 0,  0,  0,  0,  3],
       [ 0,  0,  4,  5,  6],
       [ 0,  0,  0,  7,  8],
       [ 0,  9, 10, 11, 12],
       [ 0,  0, 13,  1, 14],
       [ 0, 15, 16, 17, 18],
       [ 0,  0,  0, 19, 20],
       [ 0, 21, 22, 23, 24],
       [ 0,  0, 25, 26, 27]], dtype=int32)

### 임베딩

In [27]:
word_size = len(token.word_index) + 1
word_size

28

In [29]:
Embedding(word_size, 8)

<Embedding name=embedding_1, built=False>

In [30]:
model = Sequential()
model.add(Embedding(word_size, 8))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [31]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(padding_x, classes, epochs=20)
print(model.evaluate(padding_x, classes))

I0000 00:00:1747881040.271993    3400 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1251 MB memory:  -> device: 0, name: NVIDIA GeForce MX450, pci bus id: 0000:01:00.0, compute capability: 7.5


Epoch 1/20


I0000 00:00:1747881045.015606    3452 service.cc:152] XLA service 0x7fe598019240 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747881045.015840    3452 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce MX450, Compute Capability 7.5
2025-05-22 11:30:45.130071: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1747881045.669619    3452 cuda_dnn.cc:529] Loaded cuDNN version 90300


1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.7000 - loss: 0.6876
Epoch 2/20


I0000 00:00:1747881047.350267    3452 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 275ms/step - accuracy: 0.7000 - loss: 0.6851
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step - accuracy: 0.7000 - loss: 0.6826
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step - accuracy: 0.7000 - loss: 0.6802
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step - accuracy: 0.7000 - loss: 0.6777
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step - accuracy: 0.8000 - loss: 0.6752
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 278ms/step - accuracy: 0.8000 - loss: 0.6727
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step - accuracy: 0.8000 - loss: 0.6702
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step - accuracy: 0.8000 - loss: 0.6678
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step - accuracy: 0.8000 - loss: 0.6653
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step - accuracy: 0.8000 - loss: 0.6628
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 309ms/step - accuracy: 0.8000 - loss: 0.6603
Epoch 13/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 324ms/step - accuracy: 0.9000 - loss: 0.6578